<a href="https://colab.research.google.com/github/kshitijrajsharma/VirtuGhan/blob/master/VirtuGhan_Python_Usage_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install

Installation is easy just grab it form pip

In [ ]:
!pip install VirtuGhan==0.1.1

## Setup

Define the parameters for your computation

In [ ]:
# Define the bounding box
bbox = [83.84765625, 28.22697003891833, 83.935546875, 28.304380682962773]

# Define the parameters
start_date = "2020-01-01"
end_date = "2025-01-01" # Process five years worth of data
cloud_cover = 30
formula = "(band2-band1)/(band2+band1)"  # NDVI formula
band1 = "red"
band2 = "nir"
operation = "median"
output_dir = "virtughan_output"
timeseries = True
workers = 32 # no of parallel workers

## Cleanup
Lets clear if there are previous output in the dir

In [ ]:
import shutil , os
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

## Compute
You can run computation now , You can visualize the results or download them from the files directly after computation is done

In [ ]:
from vcube.engine import VCubeProcessor

processor = VCubeProcessor(
    bbox,
    start_date,
    end_date,
    cloud_cover,
    formula,
    band1,
    band2,
    operation,
    timeseries,
    output_dir,
)
processor.compute()

Engine starting...
Searching STAC API...
Found 68 items
Filtered 68 items that are completely within the input bounding box
Computing Band Calculation: 100%|██████████| 68/68 [02:08<00:00,  1.89s/it]
Aggregating results...
Saving aggregated result with colormap...
Creating GIF and zipping TIFF files...
Saved GIF to virtughan_output/output.gif
Saved ZIP to virtughan_output/tiff_files.zip
